# Demostración del Análisis de Carga Mutacional (TMB) con VCF

Este notebook demuestra cómo utilizar el método `calculate_tmb_analysis` de PyMutation para calcular la Carga Mutacional Tumoral (TMB) utilizando archivos VCF y generar los archivos de análisis correspondientes.

## ¿Qué es TMB?
La Carga Mutacional Tumoral (TMB) es una medida del número de mutaciones presentes en un tumor, normalizada por el tamaño del genoma interrogado. Es un biomarcador importante en oncología.


## Configuración Inicial


In [1]:
import sys
import os

# Método compatible con Jupyter notebooks para encontrar el directorio del proyecto
# Asumimos que estamos en docs/examples/analysis/ y necesitamos ir 3 niveles arriba
try:
    # Intentar usar __file__ si está disponible (scripts .py)
    script_dir = os.path.dirname(__file__)
    project_root = os.path.abspath(os.path.join(script_dir, '..', '..', '..'))
except NameError:
    # Si __file__ no está definido (Jupyter notebooks), usar el directorio actual
    # y navegar hacia arriba hasta encontrar el directorio del proyecto
    current_dir = os.getcwd()
    
    # Buscar el directorio que contiene 'src' y 'docs' (indicadores del proyecto root)
    project_root = current_dir
    while project_root != os.path.dirname(project_root):  # No hemos llegado al root del sistema
        if (os.path.exists(os.path.join(project_root, 'src')) and 
            os.path.exists(os.path.join(project_root, 'docs'))):
            break
        project_root = os.path.dirname(project_root)
    
    # Si no encontramos el proyecto, asumir que estamos en docs/examples/analysis/
    if not (os.path.exists(os.path.join(project_root, 'src')) and 
            os.path.exists(os.path.join(project_root, 'docs'))):
        project_root = os.path.abspath(os.path.join(current_dir, '..', '..', '..'))

# Configurar el path para importar pyMut
src_path = os.path.join(project_root, 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

print('✅ PYTHONPATH configurado para incluir:', src_path)
print(f'📁 Directorio del proyecto: {project_root}')

✅ PYTHONPATH configurado para incluir: /home/luisruimore/Escritorio/TFG/src
📁 Directorio del proyecto: /home/luisruimore/Escritorio/TFG


In [2]:
# Importar PyMutation y cargar datos
from pyMut.input import read_vcf
from pyMut.analysis.mutation_burden import add_tmb_method_to_pymutation

# Asegurar que el método TMB esté disponible
add_tmb_method_to_pymutation()

print('✅ Módulos importados correctamente')


✅ Módulos importados correctamente


## Cargar Datos de Ejemplo VCF


In [3]:
# Cargar el archivo VCF de ejemplo
vcf_path = os.path.join(project_root, 'src', 'pyMut', 'data', 'examples', 'VCF', 'subset_1k_variants_ALL.chr10.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased_vep_protein_gene_variant_class.vcf')

print(f'📂 Cargando archivo VCF: {vcf_path}')
py_mutation = read_vcf(
    path=vcf_path,
    create_index=True,  # Crear índice Tabix si es necesario
    cache_dir=None      # Usar directorio de caché por defecto
)

print(f"✅ Datos VCF cargados exitosamente")
print(f"📊 Forma de los datos: {py_mutation.data.shape}")
print(f"👥 Número de muestras: {len(py_mutation.samples)}")
print(f"🧬 Primeras 3 muestras: {py_mutation.samples[:3]}")


2025-07-21 02:55:27,896 | INFO | pyMut.input | Starting optimized VCF reading: /home/luisruimore/Escritorio/TFG/src/pyMut/data/examples/VCF/subset_1k_variants_ALL.chr10.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased_vep_protein_gene_variant_class.vcf
2025-07-21 02:55:27,898 | INFO | pyMut.input | Loading from cache: /home/luisruimore/Escritorio/TFG/src/pyMut/data/examples/VCF/.pymut_cache/subset_1k_variants_ALL.chr10.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased_vep_protein_gene_variant_class_493be66263d09147.parquet
2025-07-21 02:55:28,038 | INFO | pyMut.input | Cache loaded successfully in 0.14 seconds


📂 Cargando archivo VCF: /home/luisruimore/Escritorio/TFG/src/pyMut/data/examples/VCF/subset_1k_variants_ALL.chr10.shapeit2_integrated_snvindels_v2a_27022019.GRCh38.phased_vep_protein_gene_variant_class.vcf
✅ Datos VCF cargados exitosamente
📊 Forma de los datos: (1000, 2602)
👥 Número de muestras: 2555
🧬 Primeras 3 muestras: ['HG00096', 'HG00097', 'HG00099']


## Explorar las Columnas de Clasificación de Variantes

Antes de ejecutar el análisis TMB, veamos qué columnas de clasificación de variantes están disponibles en el archivo VCF:


In [4]:
# Buscar columnas de clasificación de variantes
import re

pattern = re.compile(r'^(gencode_\d+_)?variant[_]?classification$', flags=re.IGNORECASE)
variant_cols = [col for col in py_mutation.data.columns if pattern.match(col)]

print("🔍 Columnas de clasificación de variantes encontradas:")
if variant_cols:
    for i, col in enumerate(variant_cols, 1):
        print(f"  {i}. {col}")
else:
    print("  ❌ No se encontraron columnas de clasificación de variantes")

# Mostrar algunas columnas que contienen 'variant' en el nombre
variant_like_cols = [col for col in py_mutation.data.columns if 'variant' in col.lower()]
print(f"\n🔍 Columnas que contienen 'variant' ({len(variant_like_cols)}):")
for col in variant_like_cols[:5]:  # Mostrar solo las primeras 5
    print(f"  • {col}")

# Mostrar información específica de VCF
print(f"\n📋 Información del archivo VCF:")
print(f"  • Columnas totales: {len(py_mutation.data.columns)}")
print(f"  • Primeras 10 columnas: {list(py_mutation.data.columns[:10])}")


🔍 Columnas de clasificación de variantes encontradas:
  1. Variant_Classification

🔍 Columnas que contienen 'variant' (3):
  • VEP_VARIANT_CLASS
  • Variant_Classification
  • Variant_Type

📋 Información del archivo VCF:
  • Columnas totales: 2602
  • Primeras 10 columnas: ['CHROM', 'POS', 'ID', 'REF', 'ALT', 'QUAL', 'FILTER', 'HG00096', 'HG00097', 'HG00099']


## Ejecutar Análisis TMB

Ahora ejecutaremos el análisis de carga mutacional usando los datos del archivo VCF. El método generará dos archivos:
1. **TMB_analysis.tsv**: Análisis por muestra con conteos de mutaciones y TMB normalizado
2. **TMB_statistics.tsv**: Estadísticas globales (media, mediana, cuartiles, etc.)


In [5]:
# Crear directorio para los resultados
output_dir = "results_tmb_vcf"
os.makedirs(output_dir, exist_ok=True)

print(f"📁 Directorio de salida: {output_dir}")


📁 Directorio de salida: results_tmb_vcf


In [6]:
# Ejecutar el análisis TMB
print("🧬 Ejecutando análisis de carga mutacional con datos VCF...")
print("⏳ Esto puede tomar unos momentos...")

try:
    # Ejecutar análisis TMB con configuración estándar para WES
    results = py_mutation.calculate_tmb_analysis(
        genome_size_bp=60456963,  # Tamaño estándar para WES
        output_dir=output_dir,
        save_files=True
    )
    
    print("✅ Análisis TMB completado exitosamente!")
    
except Exception as e:
    print(f"❌ Error durante el análisis TMB: {e}")
    results = None


2025-07-21 02:55:28,144 | INFO | pyMut.analysis.mutation_burden | Auto-detected variant classification column: Variant_Classification


🧬 Ejecutando análisis de carga mutacional con datos VCF...
⏳ Esto puede tomar unos momentos...


2025-07-21 02:57:33,123 | INFO | pyMut.analysis.mutation_burden | TMB analysis saved to: results_tmb_vcf/TMB_analysis.tsv
2025-07-21 02:57:33,123 | INFO | pyMut.analysis.mutation_burden | TMB statistics saved to: results_tmb_vcf/TMB_statistics.tsv
2025-07-21 02:57:33,124 | INFO | pyMut.analysis.mutation_burden | Analyzed 2555 samples with 1000 total mutations
2025-07-21 02:57:33,124 | INFO | pyMut.analysis.mutation_burden | ========================================
2025-07-21 02:57:33,124 | INFO | pyMut.analysis.mutation_burden | TMB ANALYSIS SUMMARY
2025-07-21 02:57:33,124 | INFO | pyMut.analysis.mutation_burden | ========================================
2025-07-21 02:57:33,125 | INFO | pyMut.analysis.mutation_burden | • Total samples analyzed: 2555
2025-07-21 02:57:33,125 | INFO | pyMut.analysis.mutation_burden | • Average total mutations per sample: 73.6
2025-07-21 02:57:33,125 | INFO | pyMut.analysis.mutation_burden | • Average non-synonymous mutations per sample: 0.6
2025-07-21 02:

✅ Análisis TMB completado exitosamente!


## Explorar los Resultados


In [7]:
if results:
    # Obtener los DataFrames de resultados
    analysis_df = results['analysis']
    statistics_df = results['statistics']
    
    print("📊 RESULTADOS DEL ANÁLISIS TMB (VCF)")
    print("=" * 50)
    print(f"👥 Muestras analizadas: {len(analysis_df)}")
    print(f"📈 Métricas calculadas: {len(statistics_df)}")
    
    # Mostrar las primeras filas del análisis por muestra
    print("\n🔍 Primeras 5 muestras del análisis:")
    print("-" * 40)
    display(analysis_df.head())
    
else:
    print("❌ No se pudieron obtener los resultados del análisis")


📊 RESULTADOS DEL ANÁLISIS TMB (VCF)
👥 Muestras analizadas: 2555
📈 Métricas calculadas: 4

🔍 Primeras 5 muestras del análisis:
----------------------------------------


,Sample,Total_Mutations,Non_Synonymous_Mutations,TMB_Total_Normalized,TMB_Non_Synonymous_Normalized
0,HG00096,49,0,0.810494,0.000000
1,HG00097,48,0,0.793953,0.000000
2,HG00099,52,1,0.860116,0.016541
3,HG00100,70,1,1.157848,0.016541
4,HG00101,80,1,1.323255,0.016541


## Estadísticas Globales


In [8]:
if results:
    print("📈 ESTADÍSTICAS GLOBALES DEL TMB (VCF)")
    print("=" * 40)
    display(statistics_df)
    
    # Mostrar algunas estadísticas clave
    print("\n🎯 ESTADÍSTICAS CLAVE:")
    print("-" * 30)
    
    # TMB total normalizado
    tmb_total_stats = statistics_df[statistics_df['Metric'] == 'TMB_Total_Normalized'].iloc[0]
    print(f"🧬 TMB Total Normalizado:")
    print(f"   • Media: {tmb_total_stats['Mean']:.4f} mutaciones/Mb")
    print(f"   • Mediana: {tmb_total_stats['Median']:.4f} mutaciones/Mb")
    print(f"   • Rango: {tmb_total_stats['Min']:.4f} - {tmb_total_stats['Max']:.4f} mutaciones/Mb")
    
    # TMB no sinónimo normalizado
    tmb_nonsyn_stats = statistics_df[statistics_df['Metric'] == 'TMB_Non_Synonymous_Normalized'].iloc[0]
    print(f"\n🎯 TMB No Sinónimo Normalizado:")
    print(f"   • Media: {tmb_nonsyn_stats['Mean']:.4f} mutaciones/Mb")
    print(f"   • Mediana: {tmb_nonsyn_stats['Median']:.4f} mutaciones/Mb")
    print(f"   • Rango: {tmb_nonsyn_stats['Min']:.4f} - {tmb_nonsyn_stats['Max']:.4f} mutaciones/Mb")


📈 ESTADÍSTICAS GLOBALES DEL TMB (VCF)


,Metric,Count,Mean,Median,Min,Max,Q1,Q3,Std
0,Total_Mutations,2555,73.631703,70.000000,0.0,166.000000,50.000000,103.000000,35.754549
1,Non_Synonymous_Mutations,2555,0.615656,0.000000,0.0,4.000000,0.000000,1.000000,0.745798
2,TMB_Total_Normalized,2555,1.217919,1.157848,0.0,2.745755,0.827035,1.703691,0.591405
3,TMB_Non_Synonymous_Normalized,2555,0.010183,0.000000,0.0,0.066163,0.000000,0.016541,0.012336



🎯 ESTADÍSTICAS CLAVE:
------------------------------
🧬 TMB Total Normalizado:
   • Media: 1.2179 mutaciones/Mb
   • Mediana: 1.1578 mutaciones/Mb
   • Rango: 0.0000 - 2.7458 mutaciones/Mb

🎯 TMB No Sinónimo Normalizado:
   • Media: 0.0102 mutaciones/Mb
   • Mediana: 0.0000 mutaciones/Mb
   • Rango: 0.0000 - 0.0662 mutaciones/Mb


## Comparación VCF vs MAF

Este ejemplo demuestra cómo PyMutation puede procesar tanto archivos MAF como VCF para el análisis de carga mutacional:

### Ventajas del formato VCF:
- **Formato estándar**: VCF es el formato estándar para variantes genómicas
- **Información completa**: Incluye información detallada sobre genotipos y calidad
- **Compatibilidad**: Compatible con herramientas de análisis genómico estándar
- **Anotaciones VEP**: Puede incluir anotaciones de Variant Effect Predictor

### Consideraciones:
- **Procesamiento**: Los archivos VCF pueden requerir más tiempo de procesamiento inicial
- **Memoria**: Archivos VCF grandes pueden requerir más memoria
- **Caché**: PyMutation utiliza caché automático para acelerar cargas posteriores

### Uso recomendado:
- Utiliza VCF cuando tengas datos de secuenciación directos
- Utiliza MAF cuando trabajes con datos de estudios de cáncer ya procesados
- Ambos formatos producen resultados equivalentes para el análisis TMB